In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
print('ddd')

ddd


In [5]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table
import psycopg2

In [6]:
# подгружаем .env
load_dotenv()

True

In [7]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')


In [8]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [9]:
import os

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)

TABLE_NAME = 'clean_users_churn'

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000



In [10]:
import os

import psycopg
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df.head(5)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,No,0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0
2,3,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,...,No,No,No,No,Male,0,No,No,No,1
3,4,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,...,Yes,Yes,No,No,Male,0,No,No,No,0
4,5,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,...,No,No,No,No,Female,0,No,No,No,1


In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from autofeat import AutoFeatRegressor,AutoFeatClassifier
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np
import os
import mlflow
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss


In [12]:
features = [] # список признаков вашей модели
target = ['target'] # колонка с таргетом вашей модели
cat_features = [
    'paperless_billing',
    'payment_method',
    'internet_service',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies',
    'gender',
    'senior_citizen',
    'partner',
    'dependents',
    'multiple_lines',
]
num_features = ["monthly_charges", "total_charges"]

features = cat_features + num_features
split_column = "begin_date"
test_size = 0.2

df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target],  # теперь df[target] не пустой
    test_size=test_size,
    shuffle=False,
)

In [36]:
# Добавляем недостающие импорты в начале файла
import json
from datetime import datetime


FileNotFoundError: [Errno 2] No such file or directory: 'your_data.csv'

In [ ]:
         select *
            from tags
            join runs on runs.run_uuid = '403a6c9521a04b9db76588e60b26c605'
                and runs.lifecycle_stage = 'active'
                and runs.status = 'FINISHED'
                and tags.value = '403a6c9521a04b9db76588e60b26c605'
                and tags.key = 'mlflow.parentRunId'

In [21]:
import pickle  # Добавляем импорт
from mlflow.models import infer_signature

In [25]:
import os
import numpy as np
import optuna
from optuna.integration.mlflow import MLflowCallback
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    confusion_matrix,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    log_loss
)
from collections import defaultdict
import mlflow
import logging

# Настройка логов
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Проверка переменных окружени
os.environ.update({
    "MLFLOW_S3_ENDPOINT_URL": "https://storage.yandexcloud.net",
    "AWS_ACCESS_KEY_ID": os.getenv("AWS_ACCESS_KEY_ID"),
    "AWS_SECRET_ACCESS_KEY": os.getenv("AWS_SECRET_ACCESS_KEY"),
})
# Настройка MLflow
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment(EXPERIMENT_NAME)
pip_requirements = "../requirements.txt"
logger.info(f"Tracking URI: {mlflow.get_tracking_uri()}")
input_example = X_test[:10]
# Параметры эксперимента
EXPERIMENT_NAME = 'churn_denis_putov_www'
RUN_NAME = "model_bayesian_search_@2222"
STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

def objective(trial: optuna.Trial) -> float:
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5),
        "loss_function": "Logloss",
        "task_type": "CPU",
        "random_seed": 0,
        "iterations": 300,
        "verbose": False,
        "cat_features": cat_features
    }
    
    model = CatBoostClassifier(**param)
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=0)
    
    metrics = defaultdict(list)
    
    for train_index, val_index in skf.split(X_train, y_train):
        train_X = X_train.iloc[train_index]
        train_y = y_train.iloc[train_index]
        val_X = X_train.iloc[val_index]
        val_y = y_train.iloc[val_index]
        
        model.fit(
            train_X,
            train_y,
            eval_set=(val_X, val_y),
            early_stopping_rounds=20,
            verbose=False
        )
        
        probas = model.predict_proba(val_X)[:, 1]
        prediction = model.predict(val_X)
        
        tn, fp, fn, tp = confusion_matrix(val_y, prediction, normalize='all').ravel()
        
        metrics["err1"].append(fp)
        metrics["err2"].append(fn)
        metrics["auc"].append(roc_auc_score(val_y, probas))
        metrics["precision"].append(precision_score(val_y, prediction, zero_division=0))
        metrics["recall"].append(recall_score(val_y, prediction, zero_division=0))
        metrics["f1"].append(f1_score(val_y, prediction, zero_division=0))
        metrics["logloss"].append(log_loss(val_y, probas))
    
    # Агрегация метрик
    trial.set_user_attr("err1", np.mean(metrics["err1"]))
    trial.set_user_attr("err2", np.mean(metrics["err2"]))
    trial.set_user_attr("precision", np.mean(metrics["precision"]))
    trial.set_user_attr("recall", np.mean(metrics["recall"]))
    trial.set_user_attr("f1", np.mean(metrics["f1"]))
    trial.set_user_attr("logloss", np.mean(metrics["logloss"]))
    
    return np.mean(metrics["auc"])

# Основной блок выполнения
if __name__ == "__main__":
    # Создаем родительский запуск
    with mlflow.start_run(run_name=RUN_NAME) as parent_run:
        parent_run_id = parent_run.info.run_id
        
        # 4. Исправьте тег MLFLOW_PARENT_RUN_ID
        mlflc = MLflowCallback(
            tracking_uri=mlflow.get_tracking_uri(),
            metric_name="auc",
            create_experiment=True,
            mlflow_kwargs={
                'experiment_name': EXPERIMENT_NAME,
                'tags': {
                    'mlflow.parentRunId': parent_run_id,  # Правильное имя тега
                    "mlflow.runName": "optuna_trial"
                },
                'nested': True
            }
        )

        # Создание исследования Optuna
        study = optuna.create_study(
            sampler=optuna.samplers.TPESampler(),
            direction="maximize",
            study_name=STUDY_NAME,
            storage=STUDY_DB_NAME,
            load_if_exists=True
        )
        
        # Запуск оптимизации
        study.optimize(
            objective,
            n_trials=10,
            callbacks=[mlflc],
            show_progress_bar=True
        )

        # Логирование результатов в родительский запуск
        mlflow.log_params(study.best_params)
        mlflow.log_metrics({
            "best_auc": study.best_value,
            "err1": study.best_trial.user_attrs["err1"],
            "err2": study.best_trial.user_attrs["err2"]
        })
        
        # Логирование модели в родительский запуск
        best_model = CatBoostClassifier(**study.best_params, cat_features=cat_features)
        best_model.fit(X_train, y_train, verbose=False)
        # Создание поддиректории для артефактов
        artifact_path = "cv"
        os.makedirs(artifact_path, exist_ok=True)

        # 1. Сохранение модели в формате pickle
        model_pkl_path = os.path.join(artifact_path, "model.pkl")
        with open(model_pkl_path, "wb") as f:
            pickle.dump(best_model, f)

        # 2. Логирование модели через MLflow
        mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="cv",
        registered_model_name="best_churn_model",
        signature = infer_signature(X_train, best_model.predict(X_train)),
        input_example=input_example
        )
    
        mlflow.catboost.log_model(
            cb_model=best_model,
            artifact_path="cv",
            registered_model_name="best_churn_model",
            pip_requirements=pip_requirements,
            input_example=input_example,
            metadata={'model_type': 'monthly'},
            signature = infer_signature(X_train, best_model.predict(X_train))
        )
        print("Optimization complete!")
    
    # Логирование CatBoost модели


    print("Optimization complete!")

INFO:__main__:Tracking URI: http://127.0.0.1:5000
/tmp/ipykernel_372780/3242878903.py:105: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2025-04-06 18:45:51,661] Using an existing study with name 'churn_model' instead of creating a new one.


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-04-06 18:45:56,676] Trial 160 finished with value: 0.8388387017135562 and parameters: {'learning_rate': 0.039628872956278094, 'depth': 3, 'l2_leaf_reg': 1.9111495549607778, 'random_strength': 0.7059264707492978}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 18:46:01,154] Trial 161 finished with value: 0.8374116162946659 and parameters: {'learning_rate': 0.028059377926204825, 'depth': 2, 'l2_leaf_reg': 1.701165201119097, 'random_strength': 0.5569517773615004}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 18:46:05,960] Trial 162 finished with value: 0.838068143328752 and parameters: {'learning_rate': 0.060063294065786234, 'depth': 2, 'l2_leaf_reg': 2.194116537489349, 'random_strength': 0.8372374166605827}. Best is trial 41 with value: 0.8419467071459137.
[I 2025-04-06 18:46:09,500] Trial 163 finished with value: 0.8378831716822899 and parameters: {'learning_rate': 0.05049556095304804, 'depth': 5, 'l2_leaf_reg': 1.8265563690472266, 'random_stren

/home/mle-user/mle_projects/.venv_name/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'best_churn_model' already exists. Creating a new version of this model...
2025/04/06 18:46:58 INFO mlflow.tracking._model_registry.client: 

Optimization complete!
Optimization complete!


Created version '13' of model 'best_churn_model'.
